# Script content

## Here we train agan i sibgle 20 topics model on all of the data. The goal is to build a recommendation engine and we do that as foillows.

### 1) Extract tpoics distribution per document from the model object and for computng expertises on topics we sum these probabilities to folder level over all outbox emails (now it may be clear why we removed forward emails texts). Caal this value OUT_sum, derived per folder and topic 

### 2) For level of interest in a topic we firts sum over all inbox mails, at folder topic level, call this IN_sum

### 3) We consider OUT_sum to be a good measure of knowldge on a topic, as it accounts for both chances that email is indeed about a topic as wel as how often folder owner speaks about it

## 4) Interst in a topic is (considering recommendation engine as the goal!) is measured by the ration OUT_sum/IN_sum. We will use this measure in order to account for situations where extert talks to other exterts, thus needs no mentor there. 

### 5) For a mentor and mentee pair of executives (i,j) (ordered pair!) with corresponding values for a topic t, we then consider the follwing metric

#               M(t,i,j):=OUT_sum(i,t)*(OUT_sum(j,t) + beta)/(IN_sum(j,t) + beta)

### as a measure of the gfoodness of the match for topic t. Summing over all topics gives

# M(i,j)=Sum_{i,j} M(t,i,j)

### as our overall measure of match. This is in fact an inner rpoduct and in is high where the metor and mentee have both high values in many topics.

## Our recommendation engine will propose per metor say 3 mentees with maximal values of M(i,.). 

# Notice that we match based on expertise  and preference levels as well as interst in getting mentorship (according to our interpretation) over all topics jointly!!!
#### Off course further analysis will yield what are the actual topics per pair that are really actual.

# How to test this unsuperwised engine?

## Well as a complex thing like  car or plane is not tested by a single test, our model consiting of various steps alse need to be addressed in various ways. LDA topics model has already been tested so we need to test the method developed here. We propse the following three metrics.

# Method 1

### We randomly assign emails to folders, with same prior as observed across folders. That is we sample folder ids with probability equal to proportion of emaisl in it (in and outbox together. we could also refine with a two step procedure but we do it this way for now.

### We then compute the above defined metrics and at the end sum over all metors+top 3 mentees. .

### Repeating this say 5000 times will give us a distribution and we are intertsed in the p-value of observing what we observed. If it is highly unlikely (in the tail) then obviouselly our matching system is menaingfull. This permutation test type of approach is often used in applied statistics---say determining whether some quantity (height etc) in one group is significantly different then in the other.

# Method 2

### We don' t alter email folders, but instead of picking top three mentees per mentor we sample three random mentees to see wher about is our best choice. This is also an estimate of the distribution of our matching metric. 

# Method 3

### Instead of resigning emails to folders we pick random pairs of mentors as well as mentees (without replacement so that each mentor is in exactly one pair and same for metees side), and in addition we pick at random say a persentage of topic (50% or 10 topics) and swap the correspondig OUT_sum (mentors side ) and IN_sum values (mentees side), and the recompute everything including final sum per mentor metee combination and the again sum the top three values for each mentor. 

### Here too we wish to observe that out true value is very high up in that distribution. That is if one reassigns these preferances in an arbitrary way we get much lower top scores.

### Let us now get to coding.

### Load cleaned email texts 

In [1]:

import cPickle 
matrixpath = file('/notebooks/LDA models and data/Data Frames and lists/text_term_matrix_clean.pkl', 'rb')
text_term_matrix=cPickle.load(matrixpath )

matrix0path = file('/notebooks/LDA models and data/Data Frames and lists/text_term_matrix_clean.pkl', 'rb')
text_clean=cPickle.load(matrix0path)

dictfilepath=file('/notebooks/LDA models and data/Data Frames and lists/Dictionary.pkl', 'rb')
Dictionary=cPickle.load(dictfilepath)



Using TensorFlow backend.


In [5]:
import gensim
from gensim.models import ldamodel
import pandas as pd
import numpy as np

### Fit a LDA model with 20 topics

In [6]:
ldamodel_20 =gensim.models.ldamulticore.LdaMulticore(corpus=text_term_matrix, num_topics=20, id2word=Dictionary, workers=10,\
chunksize=2000, passes=1, batch=False, alpha='symmetric', eta=None, decay=0.5, offset=1.0,\
eval_every=10, iterations=50, gamma_threshold=0.001, random_state=None, minimum_probability=0.01,\
minimum_phi_value=0.01, per_word_topics=False)

### Collect inferred document topic probabilities from the model object

In [14]:
from gensim.models import ldamodel
doc_topic_prob=[]
for bowplus in text_term_matrix0:
    doc_topic_prob.append([bowplus[0] ,bowplus[1] ,ldamodel_20.get_document_topics(bowplus[2])])

### Save to dick

In [289]:
import cPickle
with open('/notebooks/LDA models and data/Data Frames and lists/doc_topic_prob.pkl', 'wb') as pickle_file:
    cPickle.dump(obj=doc_topic_prob, file=pickle_file, protocol=cPickle.HIGHEST_PROTOCOL)

### Unpac doc topic probabilities list to an array (and then data frame) with columns: 

## 'dirpath', 'inout_id','emailid', 'topicid', 'prob'

### With that file we can do everything we need to build and evaluate our recommendation engine as described above

In [283]:
k=0
resarray=np.array(['dirpath', 'inout_id','emailid' ,'topicid', 'prob'], dtype=np.dtype('a16'), ndmin=2)
for bowplus in doc_topic_prob:
    if k % 10000==0:
        print k
    comunalia=np.array([bowplus[0],bowplus[1],str(k)], ndmin=2)
    probs =np.array([list(i) for i in bowplus[2]], dtype=np.dtype('a16'))
    rep=len(bowplus[2])
    comunalia_rep=np.repeat(a=comunalia, repeats=rep, axis=0)
    thisbow=np.ma.concatenate([comunalia_rep, probs], axis=-1)
    resarray=np.ma.concatenate([resarray, thisbow], axis=0)
    k=k+1

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000


### Running of the code lines in the previous cell took very long (app. 4 hours) and without a dount this can be handleded more efficiently. However due to time contaraints we shall leave it as is for now.

In [288]:
# Save!!
pickle_file=file('/notebooks/LDA models and data/Data Frames and lists/resarray.pkl', 'wb')
cPickle.dump(obj=resarray, file=pickle_file, protocol=cPickle.HIGHEST_PROTOCOL)

### Check outpout

In [290]:
resarray[0:10,:]

masked_array(data =
 [['dirpath' 'inout_id' 'emailid' 'topicid' 'prob']
 ['hain-m' '0' '0' '0' '0.0250000000244']
 ['hain-m' '0' '0' '1' '0.0250000000762']
 ['hain-m' '0' '0' '2' '0.0250000000577']
 ['hain-m' '0' '0' '3' '0.0250000000608']
 ['hain-m' '0' '0' '4' '0.0250000000233']
 ['hain-m' '0' '0' '5' '0.0250000002241']
 ['hain-m' '0' '0' '6' '0.0250000000178']
 ['hain-m' '0' '0' '7' '0.0250000000259']
 ['hain-m' '0' '0' '8' '0.025000000075']],
             mask =
 False,
       fill_value = N/A)

### Make a data frame so we can aggregate

In [292]:
columnnames=['dirpath', 'inout_id','emailid' ,'topicid', 'prob']

df_recommend = pd.DataFrame(resarray[1:,:], columns=columnnames)


In [294]:
print df_recommend.shape
print resarray.shape

(1951525, 5)
(1951526, 5)


### Cast probabilities back to float. We needed them as chars to unpack as array doesn't accept mixed data types and we have a dirpath there

In [295]:
df_recommend['prob']=df_recommend['prob'].apply(lambda x : float(x))

In [296]:
df_recommend[0:10]

,dirpath,inout_id,emailid,topicid,prob
0,hain-m,0,0,0,0.025
1,hain-m,0,0,1,0.025
2,hain-m,0,0,2,0.025
3,hain-m,0,0,3,0.025
4,hain-m,0,0,4,0.025
5,hain-m,0,0,5,0.025
6,hain-m,0,0,6,0.025
7,hain-m,0,0,7,0.025
8,hain-m,0,0,8,0.025
9,hain-m,0,0,9,0.025


### Check precision since we are going to sum

### OK!

In [373]:
df_recommend.iloc[2345,4]

0.32674993661000001

### Add counter columns and aggregate to get IN_sum and OUT_sum as well as number of emails per folder (plus inout_id)

In [371]:
df_recommend['count_emails']= 1
df_grouped =df_recommend.groupby(['dirpath','inout_id', 'topicid'], as_index=False)[['prob', 'count_emails']].sum()

In [380]:
print df_grouped.shape
df_grouped[0:4]

(5796, 5)


,dirpath,inout_id,topicid,prob,count_emails
0,allen-p,0,0,120.038677,635
1,allen-p,0,1,64.018270,462
2,allen-p,0,10,70.678547,427
3,allen-p,0,11,212.328779,832


### OK! because 150 folders here, 20 topics, and in and out box, which is 6000 but not all folders may easily have a score for all topics above gensim threshold to return by model object as gensim code author explained in a blog

### Split frame to in and out box parts

In [389]:
df_grouped_in=df_grouped[df_grouped['inout_id']=='0']

df_grouped_out=df_grouped[df_grouped['inout_id']=='1']

### Makes sense, one value per outbox topic combination and few less as we miss few out boxes

In [385]:
print df_grouped_out.shape
print df_grouped_in.shape

(2796, 5)
(3000, 5)


In [390]:
df_grouped_out.rename(columns={'dirpath': 'mentor', 'prob': 'OUT_sum'}, inplace=True)
df_grouped_in.rename(columns={'dirpath': 'mentee', 'prob': 'IN_sum'}, inplace=True)

In [396]:
df_joined = df_grouped_out.merge(right=df_grouped_in, how='inner', left_on=['mentor', 'topicid'], right_on=['mentee', 'topicid'], suffixes=('_mentor', '_mentee'))

In [398]:
df_joined[0:4]

,mentor,inout_id_mentor,topicid,OUT_sum,count_emails_mentor,mentee,inout_id_mentee,IN_sum,count_emails_mentee
0,allen-p,1,0,20.295934,65,allen-p,0,120.038677,635
1,allen-p,1,1,4.660352,24,allen-p,0,64.018270,462
2,allen-p,1,10,0.527964,14,allen-p,0,70.678547,427
3,allen-p,1,11,24.096311,82,allen-p,0,212.328779,832


In [400]:
df_joined['INsum_OUTsum']=df_joined[['OUT_sum', 'IN_sum']].apply(lambda x: x[0]/x[1])

In [401]:
df_joined[0:4]

,mentor,inout_id_mentor,topicid,OUT_sum,count_emails_mentor,mentee,inout_id_mentee,IN_sum,count_emails_mentee,INsum_OUTsum
0,allen-p,1,0,20.295934,65,allen-p,0,120.038677,635,NaN
1,allen-p,1,1,4.660352,24,allen-p,0,64.018270,462,NaN
2,allen-p,1,10,0.527964,14,allen-p,0,70.678547,427,NaN
3,allen-p,1,11,24.096311,82,allen-p,0,212.328779,832,NaN


### Add ratio (IN_sum+beta) /(OUT_sum+beta)

In [441]:
beta=0.1
def in_over_out(x):
    return (x['IN_sum']+beta)/(x['OUT_sum']+beta)
df_joined['INsum_OUTsum']=df_joined.apply(in_over_out, axis=1)

In [442]:
df_joined[0:4]

,mentor,inout_id_mentor,topicid,OUT_sum,count_emails_mentor,mentee,inout_id_mentee,IN_sum,count_emails_mentee,INsum_OUTsum
0,allen-p,1,0,20.295934,65,allen-p,0,120.038677,635,5.890325
1,allen-p,1,1,4.660352,24,allen-p,0,64.018270,462,13.469228
2,allen-p,1,10,0.527964,14,allen-p,0,70.678547,427,112.711174
3,allen-p,1,11,24.096311,82,allen-p,0,212.328779,832,8.779387


In [443]:
df_mentors=df_joined[['mentor', 'topicid', 'OUT_sum']]
df_mentees=df_joined[['mentee', 'topicid', 'INsum_OUTsum']]

### The structure of dat ais such that there are folder topic combinations with no emails that ever had a topic probabiliity above the gensim threshold for reporting. In such cases tere is obviousely no basi for match on that topic (if either mentor or mentee has no sum variable. This remark actually applies already when defining IN_sum/OUT_sum. One may argue that such case should be handled with more care, however if one recievs emails on a topic and never sends any then it s a valid questin whther that person really has interest in this topic.

In [444]:
df_cartesian=df_mentors.merge(right=df_mentees, how='inner', left_on='topicid', right_on='topicid' ,\
                              suffixes=('_mentor', '_mentee'))


def mult_cols(x):
    return x['OUT_sum']*x['INsum_OUTsum']

df_cartesian['topic_match_score']=df_cartesian.apply(mult_cols, axis=1)

In [446]:
df_cartesian[0:4]

,mentor,topicid,OUT_sum,mentee,INsum_OUTsum,topic_match_score
0,allen-p,0,20.295934,allen-p,5.890325,119.549644
1,allen-p,0,20.295934,arnold-j,0.606127,12.301921
2,allen-p,0,20.295934,arora-h,4.811530,97.654497
3,allen-p,0,20.295934,badeer-r,3.254494,66.052998


In [454]:
df_cartesian_agg =df_cartesian.groupby(['mentor', 'mentee'], as_index=False)['topic_match_score'].sum()

# THe first result of the final part if our assignmentg is obtained now.

# The table with top 3 recomemndations for each folder (as mentor) is produced in teh next cell:

In [472]:
df_recommendation_engine=df_cartesian_agg.groupby('mentor').head(3).reset_index(drop=True)
df_recommendation_engine[0:30]

,index,mentor,mentee,topic_match_score
0,63,allen-p,linder-e,8.142937e+04
1,54,allen-p,keavey-p,4.318518e+04
2,139,allen-p,ybarbo-p,4.305624e+04
3,205,arnold-j,linder-e,6.478381e+05
4,204,arnold-j,lewis-a,4.806095e+05
5,196,arnold-j,keavey-p,3.969606e+05
6,347,arora-h,linder-e,3.150637e+04
7,338,arora-h,keavey-p,1.082759e+04
8,423,arora-h,ybarbo-p,1.068032e+04
9,489,badeer-r,linder-e,1.016079e+05


### In the remainde we collect required statistics for our 3 valuation methods

# New version with code remainder is coming tomorrow